# 🎨 Gretel - Navigator Data Designer SDK: Conversational Data Synthesis

This notebook demonstrates how to use the Gretel Navigator SDK to build a synthetic data generation pipeline step-by-step, rather than using a single YAML configuration. We will create multi-turn user-assistant dialogues tailored for fine-tuning language models. These synthetic dialogues can then be used as domain-specific training data to improve model performance in targeted scenarios.

In [ ]:
%%capture
%pip install -U gretel_client

In [ ]:
from gretel_client.navigator import DataDesigner

## ⚙️ Data Designer Configuration with the SDK

Instead of relying on a single YAML configuration file, we construct our pipeline interactively. This provides granular control over how we guide LLMs to produce realistic, domain-specific conversations. By adjusting prompts, seed data, and instructions, we can quickly iterate and refine our data generation process.

### Choosing the Model Suite
When initializing the `DataDesigner`, you can specify a `model_suite` (e.g., `apache-2.0` or `llama-3.x`). This choice determines the models and associated licenses used during data generation. Choose a suite that aligns with your compliance and licensing requirements.


In [ ]:
data_designer = DataDesigner(
    api_key="prompt",
    model_suite="apache-2.0",
)

### Setting Special System Instructions

We can provide global guidance (system instructions) to the underlying LLMs. This ensures the generated dialogues follow specific rules, maintain factual accuracy, and avoid disallowed content. These instructions will influence all subsequent generated text, helping maintain consistency and quality.

In [ ]:
data_designer.special_system_instructions = """
    You are an expert conversation designer and domain specialist. Your job is to
    produce realistic user-assistant dialogues for fine-tuning a model. Always ensure:
    - Responses are factually correct and contextually appropriate.
    - Communication is clear, helpful, and matches the complexity level.
    - Avoid disallowed content and toxicity.
    - After the two-turn conversation, provide a single toxicity assessment for the user's messages in the entire conversation.
 """


### Adding Categorical Seed Columns

We define categorical seed columns that set the context for the generated dialogues. For example, domain and topic determine what the conversation is about, while complexity guides the level of detail and difficulty. By using `num_new_values_to_generate`, we can automatically expand the range of topics or domains, increasing the diversity of generated data without manually specifying all values.

In [ ]:
data_designer.add_categorical_seed_column(
    name="domain",
    description="The domain of user assistant queries",
    values=["Tech Support", "Personal Finances", "Educational Guidance"],
    subcategories=[
        {
            "name": "topic",
            "values": {
                "Tech Support": [
                    "Troubleshooting a Laptop",
                    "Setting Up a Home Wi-Fi Network",
                    "Installing Software Updates"
                ],
                "Personal Finances": [
                    "Budgeting Advice",
                    "Understanding Taxes",
                    "Investment Strategies"
                ],
                "Educational Guidance": [
                    "Choosing a College Major",
                    "Effective Studying Techniques",
                    "Learning a New Language"
                ]
            },
            "num_new_values_to_generate": 3
        }
    ],
    num_new_values_to_generate=3
)

data_designer.add_categorical_seed_column(
    name="complexity",
    description="The complexity level of the user query",
    values=["Basic", "Intermediate", "Advanced"]
)


### Adding Generated Data Columns
We now define the columns that the model will generate. These prompts instruct the LLM to produce the actual conversation: a user query (user_message), the assistant’s response (assistant_message), a follow-up user query (user_message_2), and another assistant response (assistant_message_2). Finally, we generate a toxicity_label to assess user toxicity over the entire conversation.

You can easily modify or refine these prompt templates to adjust the style, complexity, or constraints of the generated data. Maintaining continuity and consistency across turns ensures the dialogues are realistic and useful for fine-tuning.

#### Turn 1: User Message and Assistant Response

In the first turn, we define the user_message column to simulate the user's initial query and then the assistant_message column for the assistant's reply. Ensuring that the assistant message does not always start the same way helps produce more natural variations.

In [ ]:
# User initial message
data_designer.add_generated_data_column(
    name="user_message",
    generation_prompt=(
        "The user is seeking help or information in the {domain} domain, specifically about the topic of {topic}, "
        "at a {complexity} complexity level.\n\n"
        "The user_message should:\n"
        "- Sound natural and realistic.\n"
        "- Avoid disallowed content.\n"
        "- Reflect the specified domain, topic, and complexity level.\n"
        "- Do not include headers, explanations, or assessments.\n"
        "- Do not include formatting like '## ...' or similar.\n"
    )
)

# Assistant responds
data_designer.add_generated_data_column(
    name="assistant_message",
    generation_prompt=(
        "As a helpful assistant, write a response to the user's query below:\n"
        "Query: {user_message}\n\n"
        "Instructions:\n"
        "- Provide a clear, accurate, and contextually relevant response.\n"
        "- Be correct, non-toxic, and helpful.\n"
        "- Avoid disallowed content. If the request is disallowed, provide a safe refusal.\n"
        "- To encourage variety, do not always start your response with the same phrase.\n"
        "- Only provide the message, do not add headers, explanations, or assessments.\n"
        "- Do not include formatting like '## ...' or similar.\n"
    )
)

#### Turn 2: User Follow-Up and Assistant Response
In the second turn, the user sends a follow-up message, and the assistant responds again, maintaining continuity, complexity, and context. The user’s follow-up should logically build on the previous exchange, and the assistant should reflect the given complexity level, ensuring cohesive multi-turn dialogues.

In [ ]:
# User follows up
data_designer.add_generated_data_column(
    name="user_message_2",
    generation_prompt=(
        "The user now follows up on the assistant_message.\n"
        "Previous User Query: {user_message}\n"
        "Previous Assistant Response: {assistant_message}\n\n"
        "The second user_message should:\n"
        "- Be a logical follow-up or request for clarification or more detail based on the assistant's prior response.\n"
        "- Maintain the same domain, topic, and complexity.\n"
        "- Sound natural and realistic.\n"
        "- Avoid disallowed content.\n"
        "- Do not start with 'Could you provide' or similar repetitive phrasing. Vary the question style.\n"
        "- Do not include headers, explanations, or assessments.\n"
        "- Do not include formatting like '## ...' or similar.\n"
    ),
    columns_to_list_in_prompt="all_categorical_seed_columns"
)

# Assistant responds again
data_designer.add_generated_data_column(
    name="assistant_message_2",
    generation_prompt=(
        "The user has followed up with another query:\n"
        "Previous User Query: {user_message}\n"
        "Previous Assistant Response: {assistant_message}\n"
        "New User Query: {user_message_2}\n\n"
        "Instructions:\n"
        "- Provide a clear, accurate, and contextually relevant follow-up response.\n"
        "- Maintain the complexity level and continue to be non-toxic and helpful.\n"
        "- Avoid disallowed content. If the request is disallowed, provide a safe refusal.\n"
        "- Vary your opening or start directly, do not always begin the same way.\n"
        "- Only provide the message, do not add headers, explanations, or assessments.\n"
        "- Do not include formatting like '## ...' or similar.\n"
    ),
    columns_to_list_in_prompt="all_categorical_seed_columns"
)


#### Assessment Column: Toxicity Label
For the final step, we add a toxicity assessment column to classify the overall toxicity level of the user's messages across the entire two-turn conversation. By setting llm_type='judge', we ensure that a specialized classification model is used, making it well-suited for tasks like toxicity evaluation. This helps ensure that the final dataset is safe and compliant with given standards.

In [ ]:
# Toxicity Label for the entire 2-turn conversation
# Note: We set `llm_type='judge'` to indicate we are using a classification/judgment model.

data_designer.add_generated_data_column(
    name="toxicity_label",
    generation_prompt=(
        "Analyze all user messages in this two-turn conversation and determine the overall toxicity level of the user's behavior.\n\n"
        "Consider:\n"
        "- First User Message: {user_message}\n"
        "- Assistant Response: {assistant_message}\n"
        "- Second User Message: {user_message_2}\n"
        "- Assistant Response: {assistant_message_2}\n\n"
        "Focus primarily on the user's messages to detect any disallowed content, harassment, hate, or harmful language.\n"
        "If any user message exhibits toxicity, reflect it in the final label.\n\n"
        "Possible categories:\n"
        "- \"None\": No toxicity detected in user messages.\n"
        "- \"Mild\": Slightly rude or sarcastic but not hateful or harmful.\n"
        "- \"Moderate\": Some disrespectful or harassing language.\n"
        "- \"Severe\": Overt hate, harassment, or harmful content.\n\n"
        "Provide only one label. If all user messages are polite and safe, label \"None\".\n"
        "Do not elaborate beyond providing the label."
    ),
    llm_type="judge"
)

## 👀 Generating a dataset preview

- Preview mode allows you to quickly iterate on your data design.

- Each preview generation call creates 10 records for inspection, helping you verify prompts and instructions before running a larger batch job.

In [ ]:
# Generate a preview
preview = data_designer.generate_dataset_preview(num_records=10)

In [ ]:
preview.dataset

## 🔎 Easily inspect individual records

- Run the cell below to display individual records for inspection.

- Run the cell multiple times to cycle through the 10 preview records.

- Alternatively, you can pass the `index` argument to `display_sample_record` to display a specific record.

In [ ]:
preview.display_sample_record()

## 🤔 Like what you see?

- Submit a batch workflow!

In [ ]:
batch_job = data_designer.submit_batch_workflow(num_records=25)

In [ ]:
# Check to see if the Workflow is still active.
batch_job.workflow_run_status

In [ ]:
df = batch_job.fetch_dataset(wait_for_completion=True)

In [ ]:
path = batch_job.download_evaluation_report(wait_for_completion=True)

By following these steps and leveraging the interactivity of the SDK, you can refine prompts, generate realistic dialogues, and ensure the resulting dataset is high-quality, non-toxic, and aligned with your domain-specific requirements.